# Анализ NER-модели модуля Stanza на примере записок XVIII в.

Итак, мы хотим посмотреть, насколько сильно NER-модель споткнется о данные, непохожие на современный русский язык. Сами stanza утверждают, что они используют самый подробный из доступных корпусов и их русский F1-score равен около 92.9, наравне с французским. А выше -- только голландский с F1-score ок. 94.8.

Но важно отметить, что в русской модели мы ограничены четырьмя типами именнованных сущностей: PER, LOC, ORG и MISC. У английской модели с 18 типами ИС F1-score всего лишь 88.8, однако если бы там были бы только 4 типа ИС, возможно, показатель был бы гораздо выше.

Загружаем все нужное и берем отрывок из записок конца XVIII века. В таких текстах могут найтись разные ловушки для NER. К примеру, очень сильно могут повлиять на вычисления орфография и правила вежливости ("его превосходительство" и т.д.)

In [3]:
from prozhito_tools import dump
import stanza
import re

In [4]:
dw = dump.Wrapper(csvpath='prozhito-dump/')

In [5]:
dw.notes

[ #463400 "Воскресенье. Вчера приехал..." @2950 [0-0-0] ,
  #465445 "Ночью мы достигли..." @2268 [0-0-0] ,
  #430743 "Суббота. Москва. Еду..." @795 [0-12-21] ,
  ... ,
  #31036 "a..." @82 [2959-3-9] ]

In [6]:
notes = dw.notes[(1782, 1, 1):(1790, 5, 9)]

In [7]:
notes

[ #310518 "В Корпусе был..." @1740 [1782-1-4] ,
  #310519 "За обедом у..." @1740 [1782-1-6] ,
  #310520 "Нынче, в первый..." @1740 [1782-1-9] ,
  ... ,
  #335377 "Среда. Проспав ночь..." @1850 [1789-9-10] ]

Берем первые 20 записок (этот один и тот же автор).

In [8]:
notes_text = ""
for i in range(20):
    notes_text += re.sub("<.+?/>", "", notes[i].text) + " "

In [9]:
notes_text[:200]

'В Корпусе был совет. Я находился в числе тех, которые держали караул во время этого совета. В совещаниях не происходило ничего особеннаго, только что господину Теклю велено сдать бумаги г-ну Фрейтагу,'

Прогоняем текст через русскую модель от stanza и смотрим:

In [143]:
stanza.download('ru')

2020-11-20 02:46:53 INFO: Downloading default packages for language: ru (Russian)...
2020-11-20 02:46:56 INFO: File exists: /Users/fixed/stanza_resources/ru/default.zip.
2020-11-20 02:47:03 INFO: Finished downloading models and saved to /Users/fixed/stanza_resources.


In [10]:
ner = stanza.Pipeline(lang='ru', processors='tokenize,ner')
doc = ner(notes_text)
stanza_data = []
print("entity\tstart\tend\ttype")
for sent in doc.sentences:
    for ent in sent.ents:
        stanza_data.append((ent.text,
                            (ent.start_char,
                            ent.end_char),
                            ent.type))
        print(f"{ent.text}\t{ent.start_char}\t{ent.end_char}\t{ent.type}")

2020-11-20 03:31:19 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2020-11-20 03:31:19 INFO: Use device: cpu
2020-11-20 03:31:19 INFO: Loading: tokenize
2020-11-20 03:31:19 INFO: Loading: ner
2020-11-20 03:31:22 INFO: Done loading processors!


entity	start	end	type
Корпусе	2	9	LOC
Теклю	160	165	PER
Фрейтагу	191	199	PER
Бецкаго	375	382	PER
Пурпуры	447	454	LOC
Ф.	570	572	PER
Лафон	950	955	PER
Звереву	1304	1311	PER
граф Р.	1515	1522	PER
Мелин	1582	1587	PER
Ливонец Белингсгаузен	1604	1625	PER
Пурпура	1810	1817	PER
Лафоншею	1998	2006	PER
Толстая	2013	2020	PER
Крузе	2039	2044	PER
Беклемишева	2047	2058	PER
Деболи	2089	2095	PER
Поликарпов	2097	2107	PER
Бороздин	2115	2123	PER
Окулов	2125	2131	PER
Глинской	2147	2155	PER
Бутурлин	2207	2215	PER
Рибас	2217	2222	PER
Третьяковский	2224	2237	PER
Беклемишева	2239	2250	PER
Бецкаго	2273	2280	PER
Рибас	2294	2299	PER
Трубников	2322	2331	PER
Поутру	2333	2339	PER
Рибасши	2342	2349	PER
Бригонци	2354	2362	PER
Государыня	2394	2404	PER
Рибасша	2449	2456	PER
Государыни	2486	2496	PER
Бецкий	2570	2576	PER
Государыня	2680	2690	PER
Вяземскому	2751	2761	PER
Рибасши	2989	2996	PER
Государыня	2998	3008	PER
Царскосельском дворце	3115	3136	LOC
Государыню	3168	3178	PER
Бецкий	3180	3186	PER
Кадетском Корпусе	3255	

In [11]:
stanza_data[:2]

[('Корпусе', (2, 9), 'LOC'), ('Теклю', (160, 165), 'PER')]

На первый взгляд неплохо, можно найти много иностранных фамилий, которые stanza определила как тип PER.

Но чтобы проверить, насколько она точна и полна, нужно достать данные размеченные вручную. Тот же текст, который я получил из корпуса, я разметил вручную. Достанем данные из файла:

In [33]:
with open("notes_manually.tsv", encoding="utf-8") as f:
    manually_data = []
    for line in f.read().split("\n")[:-1]:
        line_tuple = (
            line.split("\t")[0],
            (int(line.split("\t")[1]),
            int(line.split("\t")[2])),
            line.split("\t")[3]
        )
        manually_data.append(line_tuple)

Вот так выглядят эти данные, совершенно так же, как и stanza_data.

In [66]:
manually_data[:2]

[('его превосходительства', (2183, 2205), 'PER'),
 ('Царскосельском дворце', (3115, 3136), 'LOC')]

Теперь наконец посмотрим, какой на самом деле у модели F1-score:

In [106]:
from copy import copy

In [108]:
def F1_score(data):
    true_positive_count = 0
    false_values = copy(manually_data)
    for ent_s in data:
        for ent_m in manually_data:
            if ent_s[1] == ent_m[1] or (ent_s[1][0] >= ent_m[1][0] and ent_s[1][1] <= ent_m[1][1]):
                if ent_s[2] in ent_m[2]:
                    true_positive_count += 1
                    if ent_m in false_values:
                        false_values.remove(ent_m)
                    break
    precision = true_positive_count / len(data)
    recall = true_positive_count / len(manually_data)
    return (2 / (recall**(-1) + precision**(-1)), false_values)
    # фунцкия возвращает не только F1-score, но и ошибки модели.

В итоге мы получаем довольно неплохой F1-score, не сильно хуже, чем тот, который заявляет stanza:

In [109]:
F1_score(stanza_data)[0]

0.9049295774647886

Вот что stanza не учла или учла не так:

In [110]:
F1_score(stanza_data)[1]

[('его превосходительства', (2183, 2205), 'PER'),
 ('дортуаре четвертой роты', (5092, 5115), 'LOC'),
 ('«Трех Женщин»', (5916, 5929), 'MISC'),
 ('г-на Пурпуры', (442, 454), 'PER'),
 ('девицею 3.', (575, 585), 'PER'),
 ('монастырю', (1341, 1350), "('LOC', 'ORG')"),
 ('монастыря', (1986, 1995), "('LOC', 'ORG')"),
 ('дворец', (3604, 3610), 'LOC'),
 ('Италианских', (4544, 4555), 'MISC'),
 ('К.', (5690, 5692), 'PER'),
 ('Рибаса', (5793, 5799), 'PER'),
 ('Бецкаго', (7631, 7638), 'PER'),
 ('монастыря', (8775, 8784), 'LOC'),
 ('Русских', (8896, 8903), 'MISC'),
 ('С.', (9453, 9455), 'PER'),
 ('Т.', (9734, 9736), 'PER'),
 ('Э.', (9737, 9739), 'PER'),
 ('П.', (9740, 9742), 'PER'),
 ('У.', (9743, 9745), 'PER'),
 ('3', (9912, 9913), 'PER'),
 ('Государыня', (9915, 9925), 'PER'),
 ('3.', (10510, 10512), 'PER'),
 ('З.', (10668, 10670), 'PER'),
 ('Б.', (10673, 10675), 'PER'),
 ('монастырь', (10730, 10739), 'LOC'),
 ('девицу П. Г. Т.', (11063, 11078), 'PER'),
 ('графе Роб.', (11413, 11423), 'PER'),
 ('г

Во-первых, stanza отказывалась учитывать слово "монастырь", хотя каждый раз в тексте имелся в виду один и тот же монастырь. Вероятно, большая проблема в том, что stanza смотрит только на предложения и не учитывает весь текст (это же подтверждается тем, что бедный Бецкий несколько раз оказался местом, хотя вроде бы угадывался и как человек).

Во-вторых, у stanzы плохо с произведениями искусства: не опознаны Дон Жуан и Севильский Цирюльник, что скорее всего связано с тем, что конкретные названия разных произведений встречаются в корпусе редко и их труднее опознать.

В-третьих, вполне вероятно, что моя разметка могла бы быть гораздо корректнее, но меня отчасти успокаивает, что F1-score довольно высокий.

В-четвертых, интересно было бы посмотреть на то, как сильно влияют на модель строчные буквы. Посмотрим, что будет, если все символы привести к нижнему регистру:

In [103]:
doc = ner(notes_text.lower())
stanza_data_lower = []
print("entity\tstart\tend\ttype")
for sent in doc.sentences:
    for ent in sent.ents:
        stanza_data_lower.append((ent.text,
                            (ent.start_char,
                            ent.end_char),
                            ent.type))
        print(f"{ent.text}\t{ent.start_char}\t{ent.end_char}\t{ent.type}")

entity	start	end	type
вяземскому	2751	2761	MISC
италианских	4544	4555	MISC
русских	8896	8903	MISC
польской	9111	9119	MISC
русскаго	14540	14548	MISC


Будет очень-очень плохо, что на самом деле тоже большая проблема. Если мы хотим для корпуса сохранить орфографию человека, который пишет в дневнике только с прописной, то stanza легко споткнется и об это.

In [105]:
F1_score(stanza_data_lower)[0]

0.020547945205479454

*Миша Сонькин*